<a href="https://colab.research.google.com/github/lgregs/TCC-CD-USP/blob/main/use_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3
import pandas as pd

In [2]:
db_file = '/content/drive/MyDrive/TCC/balancos.sqlite'

try:
    conn = sqlite3.connect(db_file)
    print(f"Conexão bem-sucedida ao banco de dados: {db_file}")

    # Carregar todos os pares (CompanyCode, Year) para o Python
    # GROUP BY CompanyCode, Year garante que temos apenas um registro por empresa por ano
    query = """
    SELECT CompanyCode, Year
    FROM balancos_financeiros
    GROUP BY CompanyCode, Year;
    """
    df_company_years = pd.read_sql_query(query, conn)

    # Desconecta do banco de dados após carregar o necessário para liberar recursos
    conn.close()
    print("Dados CompanyCode e Year carregados com sucesso. Conexão SQL fechada.")

except Exception as e:
    print(f"ERRO: Não foi possível conectar ao banco de dados ou carregar dados iniciais: {e}")
    # Certifique-se de que a conexão é fechada mesmo em caso de erro
    if 'conn' in locals() and conn:
        conn.close()
    exit() # Interrompe a execução se houver um erro crítico aqui

# 2. Encontrar o range de anos para iterar
min_global_year = df_company_years['Year'].min()
max_global_year = df_company_years['Year'].max()

# Estrutura para armazenar TODOS os resultados qualificados
all_qualified_periods = []

# Agrupa os anos por empresa em um set ANTES dos loops para evitar recalculo repetitivo
company_year_sets = df_company_years.groupby('CompanyCode')['Year'].apply(set)

print(f"\nIniciando busca por períodos contínuos entre {min_global_year} e {max_global_year}...")

# Iterar sobre todos os possíveis anos de início
for start_year in range(min_global_year, max_global_year + 1):
    # Iterar sobre todos os possíveis anos de fim, a partir do ano de início
    for end_year in range(start_year, max_global_year + 1):
        period_length = end_year - start_year + 1

        # Filtro: queremos apenas períodos com mais de 5 anos
        if period_length <= 5:
            continue

        current_period_years = set(range(start_year, end_year + 1))

        qualified_companies = []
        for company, years_present in company_year_sets.items():
            # Verifica se o período atual está contido nos anos presentes da empresa
            if current_period_years.issubset(years_present):
                qualified_companies.append(company)

        num_qualified_companies = len(qualified_companies)

        if num_qualified_companies > 0: # Armazena apenas períodos com pelo menos 1 empresa
            all_qualified_periods.append({
                'start_year': start_year,
                'end_year': end_year,
                'period_length': period_length,
                'num_companies': num_qualified_companies,
                'companies': qualified_companies
            })

# 3. Classificar os períodos encontrados
# Critério de ordenação: Primeiro por número de empresas (decrescente), depois por comprimento do período (decrescente)
all_qualified_periods.sort(key=lambda x: (x['num_companies'], x['period_length']), reverse=True)

# Supondo que 'all_qualified_periods' já foi gerada pelo código anterior
# e contém os resultados da análise de períodos contínuos.

long_periods_with_many_companies = [
    period for period in all_qualified_periods
    if period['period_length'] >= 15 and period['num_companies'] >= 50
]

# Opcional: Reordenar, embora 'all_qualified_periods' já esteja ordenada
# long_periods_with_many_companies.sort(key=lambda x: (x['num_companies'], x['period_length']), reverse=True)

print(f"\n--- Períodos Contínuos com Mais de 10 Anos e Mais de 100 Empresas ---")

if long_periods_with_many_companies:
    print(f"Total de {len(long_periods_with_many_companies)} períodos encontrados que atendem aos critérios.")

    # Imprimir os top N períodos (você pode ajustar N aqui)
    top_n_periods_to_show = 5 # Mostra os 5 melhores desses períodos filtrados

    for i, period in enumerate(long_periods_with_many_companies[:top_n_periods_to_show]):
        print(f"\n--- Período {i+1} ---")
        print(f"Período: **{period['start_year']} a {period['end_year']}** (Duração: **{period['period_length']} anos**)")
        print(f"Número de Empresas: **{period['num_companies']}**")

        # Exemplo de Empresas (limita a exibição para não sobrecarregar)
        if len(period['companies']) > 5:
            print(f"Exemplo de Empresas: {period['companies'][:5]} e mais {len(period['companies']) - 5} empresas.")
        else:
            print(f"Empresas: {period['companies']}")
else:
    print("Nenhum período contínuo encontrado com mais de 15 anos e mais de 100 empresas.")

Conexão bem-sucedida ao banco de dados: /content/drive/MyDrive/TCC/balancos.sqlite
Dados CompanyCode e Year carregados com sucesso. Conexão SQL fechada.

Iniciando busca por períodos contínuos entre 2008 e 2024...

--- Períodos Contínuos com Mais de 10 Anos e Mais de 100 Empresas ---
Total de 6 períodos encontrados que atendem aos critérios.

--- Período 1 ---
Período: **2010 a 2024** (Duração: **15 anos**)
Número de Empresas: **293**
Exemplo de Empresas: ['ACSEL', 'ADEL', 'ADESE', 'AEFES', 'AFYON'] e mais 288 empresas.

--- Período 2 ---
Período: **2008 a 2024** (Duração: **17 anos**)
Número de Empresas: **227**
Exemplo de Empresas: ['ADEL', 'AEFES', 'AFYON', 'AGHOL', 'AGYO'] e mais 222 empresas.

--- Período 3 ---
Período: **2008 a 2023** (Duração: **16 anos**)
Número de Empresas: **227**
Exemplo de Empresas: ['ADEL', 'AEFES', 'AFYON', 'AGHOL', 'AGYO'] e mais 222 empresas.

--- Período 4 ---
Período: **2009 a 2024** (Duração: **16 anos**)
Número de Empresas: **227**
Exemplo de Empres

In [7]:
import sqlite3
import pandas as pd

db_file = '/content/drive/MyDrive/TCC/balancos.sqlite'
conn = sqlite3.connect(db_file)

# Find target companies
target_period = next(
    (p for p in all_qualified_periods if p['start_year'] == 2008 and p['end_year'] == 2024),
    None
)

if not target_period:
    raise ValueError("Período 2 (2008-2024) not found!")

companies_tuple = tuple(target_period['companies'])
years_tuple = tuple(range(2008, 2025))

query = f"""
SELECT *
FROM balancos_financeiros
WHERE CompanyCode IN {companies_tuple}
AND Year IN {years_tuple};
"""

df_period2 = pd.read_sql_query(query, conn)

# 🚀 Export directly to CSV
df_period2.to_csv('balancos_periodo_2008_2024.csv', index=False)

conn.close()

print("CSV export complete: balancos_periodo_2008_2024.csv")


CSV export complete: balancos_periodo_2008_2024.csv
